In [8]:
! nvidia-smi

Wed Jun  1 08:59:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    33W /  70W |   1988MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
!pip install pytorch-tabnet optuna 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score,classification_report
import optuna as opt
import torch
import os
import joblib

In [11]:
def make_save_cv_model(i,model_name,model,best_params,optim,output_path="./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/cross_validated_models"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))
    else:
        os.mkdir(os.path.join(output_path,f"{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))

In [12]:
def train(fold_dict,fold,model_name,sc_df,tar_col,optim,optim_trial,k_folds=10,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    x = sc_df.drop([tar_col],axis=1)
    model_name = model_name 
    def objective(trial):
      train_index = fold_dict[fold]["train"]
      test_index = fold_dict[fold]["test"]
      clf = TabNetClassifier(n_d=trial.suggest_int("n_d", 8, 64),
                              n_a =trial.suggest_int("n_a", 8, 64),
                              n_steps = trial.suggest_int("n_steps",3,10),
                              gamma =trial.suggest_float("gamma", 1.0, 2.0),
                              n_independent = trial.suggest_int("n_independent",1,5),
                              n_shared = trial.suggest_int("n_shared",1,5),
                              momentum = trial.suggest_float("momentum", 0.01, 0.4),
                              optimizer_fn = torch.optim.Adam,
                              # scheduler_fn = torch.optim.lr_scheduler,
                              # scheduler_params = {"gamma" :trial.suggest_float("sch-gamma", 0.5, 0.95), "step_size": trial.suggest_int("sch_step_size", 10, 20, 2)},
                              verbose = verbose,
                              device_name = "auto"
                              )
      # print(f" train_index :: {train_index}")
      # print(f" test_index :: {test_index}")
      X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
      # print(X_train.shape, X_test.shape)
      X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
      Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
      Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
      print(Y_train.shape, Y_test.shape)
      clf.fit(X_train, Y_train,
              eval_set=[(X_test, Y_test)],
              eval_metric=['accuracy'])
      Y_pred = clf.predict(X_test)
      print(classification_report(Y_test, Y_pred, labels=[x for x in range(6)]))
      acc = accuracy_score(Y_pred, Y_test)
      return acc

    print(f"Starting optimization for fold : [{fold}/{k_folds}]")
    study = opt.create_study(direction='maximize')
    study.optimize(objective, n_trials=optim_trial)
    best_params = study.best_params
    print(f" Best params for fold : [{fold}/{k_folds}]")
    print(best_params)
    joblib.dump(best_params,f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/comp/fold_{fold}_best_params.z")
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/fold_{fold}_best_params.txt", "w+") as file:file.write(str(best_params))
    print(f"Saved best_params at : outputs/{model_name}/best_params/fold_{fold}_best_params.txt")
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = TabNetClassifier(**study.best_params)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    clf_report = classification_report(Y_test, Y_pred, labels=[x for x in range(6)])
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/classification_report/{model_name}_{fold}_classification_report.txt","w+") as file:file.write(str(clf_report))
    accuracy = accuracy_score(Y_pred, Y_test)
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/{model_name}_{fold}_accuracy_score.txt","w+") as file:file.write(f" accuracy :: {str(accuracy)}")
    try:
        print("[++] Saving the model and parameters in corresponding directories")
        make_save_cv_model(fold,model_name,clf_model,best_params,optim=optim)
    except:
        print("[-] Failed to save the model")

In [15]:
use_df = pd.read_csv("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/data/trainable_scaled_balanced.csv")
tar_col = "PCE_categorical"
model_name = "pytorch_tabnet"
optimizer = "Adam"
fold_dict = joblib.load("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/inputs/fold_vals/fold_data.z")
fold = 2

In [16]:
train(fold_dict = fold_dict,
      fold = fold,
      model_name=model_name,
      sc_df=use_df,
      tar_col=tar_col,
      optim=optimizer,
      optim_trial = 15)
print(f"[++] Ended the training process for fold {fold}")

[I 2022-06-01 08:59:30,876] A new study created in memory with name: no-name-677d8f9a-6ba3-4b0b-89cc-58fcdabc0354


Starting optimization for fold : [2/10]
Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.37446 | val_0_accuracy: 0.51706 |  0:00:10s
epoch 1  | loss: 0.90234 | val_0_accuracy: 0.6237  |  0:00:18s
epoch 2  | loss: 0.82485 | val_0_accuracy: 0.6291  |  0:00:20s
epoch 3  | loss: 0.77242 | val_0_accuracy: 0.66278 |  0:00:23s
epoch 4  | loss: 0.73707 | val_0_accuracy: 0.66926 |  0:00:26s
epoch 5  | loss: 0.7262  | val_0_accuracy: 0.67163 |  0:00:29s
epoch 6  | loss: 0.70976 | val_0_accuracy: 0.68761 |  0:00:32s
epoch 7  | loss: 0.69685 | val_0_accuracy: 0.69624 |  0:00:34s
epoch 8  | loss: 0.68921 | val_0_accuracy: 0.66213 |  0:00:37s
epoch 9  | loss: 0.68039 | val_0_accuracy: 0.70509 |  0:00:40s
epoch 10 | loss: 0.6757  | val_0_accuracy: 0.68826 |  0:00:42s
epoch 11 | loss: 0.66988 | val_0_accuracy: 0.70488 |  0:00:45s
epoch 12 | loss: 0.66387 | val_0_accuracy: 0.68243 |  0:00:48s
epoch 13 | loss: 0.65537 | val_0_accuracy: 0.71136 |  0:00:51s
epoch 14 | loss: 0.64836 | val_0_accuracy:

[I 2022-06-01 09:03:27,707] Trial 0 finished with value: 0.7944732297063903 and parameters: {'n_d': 51, 'n_a': 51, 'n_steps': 4, 'gamma': 1.373168812930887, 'n_independent': 3, 'n_shared': 3, 'momentum': 0.1015912238579528}. Best is trial 0 with value: 0.7944732297063903.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       825
           1       0.63      0.61      0.62       825
           2       0.85      0.91      0.88       823
           3       1.00      1.00      1.00       779
           4       0.55      0.57      0.56       761
           5       0.70      0.63      0.66       619

    accuracy                           0.79      4632
   macro avg       0.79      0.79      0.79      4632
weighted avg       0.79      0.79      0.79      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.16749 | val_0_accuracy: 0.606   |  0:00:02s
epoch 1  | loss: 0.86227 | val_0_accuracy: 0.62932 |  0:00:04s
epoch 2  | loss: 0.81637 | val_0_accuracy: 0.63558 |  0:00:06s
epoch 3  | loss: 0.76698 | val_0_accuracy: 0.66883 |  0:00:08s
epoch 4  | loss: 0.73052 | val_0_accuracy: 0.68048 |  0:00:10s
epoch 5  | loss: 0.72179 | val_0_accuracy: 0.68264 |  0:00:12s
epoch 6  | loss: 0.70995 | val_0_accuracy: 

[I 2022-06-01 09:06:34,730] Trial 1 finished with value: 0.7871329879101899 and parameters: {'n_d': 26, 'n_a': 51, 'n_steps': 4, 'gamma': 1.1541518460016769, 'n_independent': 2, 'n_shared': 2, 'momentum': 0.38281071283129436}. Best is trial 0 with value: 0.7944732297063903.


...

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       825
           1       0.51      0.46      0.48       825
           2       0.77      0.82      0.79       823
           3       1.00      1.00      1.00       779
           4       0.44      0.62      0.51       761
           5       0.67      0.35      0.46       619

    accuracy                           0.72      4632
   macro avg       0.73      0.71      0.71      4632
weighted avg       0.73      0.72      0.72      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 2.22294 | val_0_accuracy: 0.43351 |  0:00:06s
epoch 1  | loss: 1.26075 | val_0_accuracy: 0.51317 |  0:00:13s
epoch 2  | loss: 1.18698 | val_0_accuracy: 0.56671 |  0:00:20s
epoch 3  | loss: 0.96858 | val_0_accuracy: 0.587   |  0:00:27s
epoch 4  | loss: 0.87435 | val_0_accuracy: 0.62457 |  0:00:34s
epoch 5  | loss: 0.8009  | val_0_accuracy: 0.64767 |  0:00:41s
epoch 6  | loss: 0.78162 | val_0_accuracy: 

[I 2022-06-01 10:00:42,389] Trial 13 finished with value: 0.7355354058721935 and parameters: {'n_d': 62, 'n_a': 37, 'n_steps': 8, 'gamma': 1.6272872516453727, 'n_independent': 5, 'n_shared': 4, 'momentum': 0.1693352847966804}. Best is trial 11 with value: 0.8065630397236615.


              precision    recall  f1-score   support

           0       0.99      1.00      1.00       825
           1       0.52      0.50      0.51       825
           2       0.82      0.78      0.80       823
           3       1.00      1.00      1.00       779
           4       0.46      0.62      0.53       761
           5       0.67      0.45      0.54       619

    accuracy                           0.74      4632
   macro avg       0.74      0.72      0.73      4632
weighted avg       0.75      0.74      0.74      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 3.62727 | val_0_accuracy: 0.29447 |  0:00:05s
epoch 1  | loss: 1.59169 | val_0_accuracy: 0.36788 |  0:00:11s
epoch 2  | loss: 1.47685 | val_0_accuracy: 0.32211 |  0:00:17s
epoch 3  | loss: 1.32925 | val_0_accuracy: 0.46136 |  0:00:22s
epoch 4  | loss: 1.29697 | val_0_accuracy: 0.54145 |  0:00:28s
epoch 5  | loss: 1.13888 | val_0_accuracy: 0.58787 |  0:00:34s
epoch 6  | loss: 0.88474 | val_0_accuracy: 

[I 2022-06-01 10:10:10,061] Trial 14 finished with value: 0.7767702936096719 and parameters: {'n_d': 35, 'n_a': 17, 'n_steps': 10, 'gamma': 1.5587692846959118, 'n_independent': 1, 'n_shared': 5, 'momentum': 0.2997646055972025}. Best is trial 11 with value: 0.8065630397236615.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       825
           1       0.61      0.58      0.59       825
           2       0.83      0.91      0.87       823
           3       1.00      1.00      1.00       779
           4       0.50      0.53      0.51       761
           5       0.68      0.60      0.64       619

    accuracy                           0.78      4632
   macro avg       0.77      0.77      0.77      4632
weighted avg       0.78      0.78      0.78      4632

 Best params for fold : [2/10]
{'n_d': 60, 'n_a': 64, 'n_steps': 9, 'gamma': 1.2273719163448753, 'n_independent': 1, 'n_shared': 5, 'momentum': 0.18590202624950367}
Saved best_params at : outputs/pytorch_tabnet/best_params/fold_2_best_params.txt
Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 2.09037 |  0:00:04s
epoch 1  | loss: 1.18932 |  0:00:09s
epoch 2  | loss: 1.12224 |  0:00:14s
epoch 3 

Fold 0 has started running on 20-05-22 


Fold 0 has completed sucessfully on 17:00 20-05-22

Fold 1 has started running at 15:15 21-05-22

Fold 2 has started running at 09:45 22-05-22

Fold 2 has completed sucessfully on 10:58 22-05-22

Fold 3 has started running at 18:40 22-05-22

Fold 3 has completed sucessfully on 22-05-22

Fold 4 completed sucessfully on 21:04 on 22-05-22

Fold 5 started at 18:21 on 23-05-22

Fold 5 completed sucessfully on 19:44 on 23-05-22

Fold 6 started at 12:53 on 24-05-22

Fold 6 has completed at 14:14 on 24-05-22

Fold 7 started at 14:18 on 24-05-22

Fold 7 execution failed due to colab gpu time limit

Fold 7 trial 1 started at 11:00 on 25-05-22

Fold 7 has completed sucessfully at 12:14 on 25-05-22 

Fold 8 has started at 9:38 on 26-05-22

Fold 8 filed due to interrupted internet connection

Fold 8 trial 1 started at 13:38 on 26-05-22

Fold 8 has successfully executed at 15:33 on 26-05-22

Fold 9 has started at 13:35 on 27-05-22

Fold 9 has completed at 14:55 on 27-05-22

Editing with rectified dataset witout duplicacy because of space values

Fold 0 started at 13:21 on 28-05-22

Fold 0 completed sucessfully at 14:46 on 28-05-22

Fold 1 Failed to run due to some index error

_____ Restarting the training process again due to data distribution failure____



Fold 0 started at 10:47 on 30-05-22

Fold 0 completed successfully at 12:30 on 30-05-22

Fold 1 started at 8:27 on 31-05-22

Fold 1 execution failed due to runtime disconnection

Fold 1 started again at 9:38 on 31-05-22

Fold 1 execution failed due to gpu disconnect 

Fold 1 started again at 8:36 on 1-06-22

Fold 1 execution failed due to network disconnection

Fold 1 started again at 13:11 on 01-06-22

Fold 1 has succesfully executed at 14:25 on 01-06-22

Fold 2 started at 14:29 on 01-06-22

Fold 2 completed succesfully at 16:00 on 01-06-22